In [9]:
import pulp
import csv
from pulp import *

In [10]:
'''
Get adj_matrix from txt files.

Inputs:
filename - filename of a comma separated value file

Output:
adj_matrix - n*n matrix for n vertices where adj_matrix[i][j] represents a directed edge from i to j if its 1
'''
def read_txt(filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        adj_matrix = list(reader)
    adj_matrix = [[int(j) for j in i[0:(len(i))]] for i in adj_matrix]
#     sinks = [any(row) for row in adj_matrix]
#     sinks = [i for i, v in enumerate(sinks) if not v]
#     adj_matrix_t = list(map(list, zip(*adj_matrix)))
#     sources = [any(row) for row in adj_matrix_t]
#     sources = [i for i, v in enumerate(sources) if not v]
    return adj_matrix

In [11]:
'''
Helper utility to preprocess our input graph

Inputs:
adj_matrix - n*n matrix for n vertices where adj_matrix[i][j] represents a directed edge from i to j if its 1
sources - list of vertices that are sources eg: [0,2] where 0,2 would be valid rows and columns in adj_matrix
sinks - list of vertices that are sinks eg: [3]

Ouputs:
adj_matrix - A new adj_matrix where a super source and super sink are added to handle multiple source/sink case
edge_cap_facs - A list of variables representing edge capacity scaling factors (Our decision variable x)
                Values would be of the form x_i_j which denotes capacity scaling factor for edge going from vertex i to j
edge_flow_vals - A list of variables representing edge flow values (Variable y in LP)
                 Values would be of the form y_i_j which denotes flow for edge going from vertex i to j
vertex_dict - A list of dictionaries of form [{i:[],o:[]},{i:[],o:[]},..] 
              where i and o are input and output edges for that vertex (represented by index in this list)
              the edges here would just be the corresponding edge flow variables defined in the edge_flow_vals list
              so that this can be directly used to create flow conservation constraints in the LP
'''
def preprocess_graph(adj_matrix, sources, sinks):
    n = len(adj_matrix) #Infer the size of adj_matrix
    m = n+2 #New size of adjacency matrix as we add 2 new nodes, 1 super source and 1 super sink
    
    #Initialize the new_adj_matrix with zeros
    new_adj_matrix = []
    for i in range(m):
        new_adj_matrix.append([])
        for j in range(m):
            new_adj_matrix[i].append(0)
    #Copy over values from the original adj_matrix, 
    #basically the ith vertex in original matrix would become the (i+1)th vertex in the new matrix
    for i in range(n):
        new_adj_matrix[i+1][1:m-1] = adj_matrix[i]
    
    #Connect sources and sinks to new super source and super sink
    #0 would always be the super source and m would always be the super sink
    for s in sources:
        new_adj_matrix[0][s+1] = 1

    for t in sinks:
        new_adj_matrix[t+1][m-1] = 1
        
    #The new adjacency matrix is created. 
    #Now we need to create the edge capacity factors, edge flow factors and vertex dict
    edge_capacity_factors = []
    edge_flows = []
    vertex_dict = []
    #Initialize vertex dict
    for i in range(len(new_adj_matrix)):
        vertex_dict.append({'i':[],'o':[]})
        
    for i in range(len(new_adj_matrix)):
        for j in range(len(new_adj_matrix[i])):
            if(new_adj_matrix[i][j] == 1):
                ecf = 'x_'+str(i)+'_'+str(j)
                ef = 'y_'+str(i)+'_'+str(j)
                edge_capacity_factors.append(ecf)
                edge_flows.append(ef)
                #Outgoing for i
                vertex_dict[i]['o'].append(ef)
                #Incoming for j
                vertex_dict[j]['i'].append(ef)
                
    return new_adj_matrix, edge_capacity_factors, edge_flows, vertex_dict

In [12]:
'''
Cost function

Inputs:
x       - capacity of edge
edge    - edge indice

Outputs:
y       - cost of assigning capacity of an edge to x
'''
def cost_function(x, edge):
    return x-1


In [13]:
'''
Main Input to the program
adj_matrix - n*n matrix represented as a list of lists in python as shown below, 1 represents an edge from i->j
sources - list of vertices that are sources
sinks - list of vertices that are sinks
flow_val - Value of flow that we want to reinstate

Example

adj_matrix = [[0,1,0,0],[0,0,1,1],[0,0,0,0],[0,0,1,0]]
sources = [0]
sinks = [2]
flow_val = 2
'''
# adj_matrix = [[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0], 
# [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], 
# [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0], 
# [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0], 
# [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0], 
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], 
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], 
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
adj_matrix = read_txt('Adj_Matrix0.txt')
sources = [0,1,2]
sinks = [9,10,11]
flow_val = 7

In [14]:
#Preprocess graph, prepare variables and vertex dictionary for LP program
transition_matrix,edge_capacity_factors,edge_flows,vertex_dict = preprocess_graph(adj_matrix,sources,sinks)
source = 0 #super source
sink = len(transition_matrix)-1 #super sink

In [15]:
'''
LP Problem formulation
'''
lp_problem = pulp.LpProblem("Graph min cost", pulp.LpMinimize)
edge_cap_vars = LpVariable.dicts("EdgeCaps",edge_capacity_factors,1)
edge_flow_vars = LpVariable.dicts("EdgeFlows",edge_flows,0)

#Objective function
lp_problem += lpSum([(edge_cap_vars[i] - 1) for i in edge_capacity_factors]), "Z"

#Flow conservation constraints for nodes besides source and sink
for i in range(len(vertex_dict)):
    if(i != source and i != sink):
        lp_problem += lpSum([edge_flow_vars[j] for j in vertex_dict[i]['i']]) >= lpSum([edge_flow_vars[k] for k in vertex_dict[i]['o']])
        lp_problem += lpSum([edge_flow_vars[j] for j in vertex_dict[i]['i']]) <= lpSum([edge_flow_vars[k] for k in vertex_dict[i]['o']])

#Flow conservation for source
lp_problem += lpSum([edge_flow_vars[j] for j in vertex_dict[source]['o']]) >= flow_val
lp_problem += lpSum([edge_flow_vars[j] for j in vertex_dict[source]['o']]) <= flow_val

#Flow conservation for sink
lp_problem += lpSum([edge_flow_vars[j] for j in vertex_dict[sink]['i']]) >= flow_val
lp_problem += lpSum([edge_flow_vars[j] for j in vertex_dict[sink]['i']]) <= flow_val

#Capacity constraints
for cap, flow in zip(edge_capacity_factors, edge_flows):
    lp_problem += edge_flow_vars[flow] <= edge_cap_vars[cap]

In [16]:
# lp_problem

In [17]:
lp_problem.solve()

1

In [18]:
pulp.LpStatus[lp_problem.status]

'Optimal'

In [19]:
for variable in lp_problem.variables():
    print("{} = {}".format(variable.name, variable.varValue))

EdgeCaps_x_0_1 = 2.0
EdgeCaps_x_0_2 = 3.0
EdgeCaps_x_0_3 = 2.0
EdgeCaps_x_10_13 = 1.0
EdgeCaps_x_11_13 = 4.0
EdgeCaps_x_12_13 = 2.0
EdgeCaps_x_1_4 = 1.0
EdgeCaps_x_1_5 = 1.0
EdgeCaps_x_2_4 = 1.0
EdgeCaps_x_2_5 = 1.0
EdgeCaps_x_2_6 = 1.0
EdgeCaps_x_3_5 = 1.0
EdgeCaps_x_3_6 = 1.0
EdgeCaps_x_4_5 = 1.0
EdgeCaps_x_4_7 = 2.0
EdgeCaps_x_5_7 = 2.0
EdgeCaps_x_5_9 = 1.0
EdgeCaps_x_6_5 = 1.0
EdgeCaps_x_6_9 = 2.0
EdgeCaps_x_7_10 = 1.0
EdgeCaps_x_7_11 = 3.0
EdgeCaps_x_9_11 = 1.0
EdgeCaps_x_9_12 = 2.0
EdgeFlows_y_0_1 = 2.0
EdgeFlows_y_0_2 = 3.0
EdgeFlows_y_0_3 = 2.0
EdgeFlows_y_10_13 = 1.0
EdgeFlows_y_11_13 = 4.0
EdgeFlows_y_12_13 = 2.0
EdgeFlows_y_1_4 = 1.0
EdgeFlows_y_1_5 = 1.0
EdgeFlows_y_2_4 = 1.0
EdgeFlows_y_2_5 = 1.0
EdgeFlows_y_2_6 = 1.0
EdgeFlows_y_3_5 = 1.0
EdgeFlows_y_3_6 = 1.0
EdgeFlows_y_4_5 = 0.0
EdgeFlows_y_4_7 = 2.0
EdgeFlows_y_5_7 = 2.0
EdgeFlows_y_5_9 = 1.0
EdgeFlows_y_6_5 = 0.0
EdgeFlows_y_6_9 = 2.0
EdgeFlows_y_7_10 = 1.0
EdgeFlows_y_7_11 = 3.0
EdgeFlows_y_9_11 = 1.0
EdgeFlows_y_9_

In [20]:
# for v in edge_cap_vars:
#     print(v)
    
lp_problem.variables

<bound method LpProblem.variables of Graph min cost:
MINIMIZE
1*EdgeCaps_x_0_1 + 1*EdgeCaps_x_0_2 + 1*EdgeCaps_x_0_3 + 1*EdgeCaps_x_10_13 + 1*EdgeCaps_x_11_13 + 1*EdgeCaps_x_12_13 + 1*EdgeCaps_x_1_4 + 1*EdgeCaps_x_1_5 + 1*EdgeCaps_x_2_4 + 1*EdgeCaps_x_2_5 + 1*EdgeCaps_x_2_6 + 1*EdgeCaps_x_3_5 + 1*EdgeCaps_x_3_6 + 1*EdgeCaps_x_4_5 + 1*EdgeCaps_x_4_7 + 1*EdgeCaps_x_5_7 + 1*EdgeCaps_x_5_9 + 1*EdgeCaps_x_6_5 + 1*EdgeCaps_x_6_9 + 1*EdgeCaps_x_7_10 + 1*EdgeCaps_x_7_11 + 1*EdgeCaps_x_9_11 + 1*EdgeCaps_x_9_12 + -23
SUBJECT TO
_C1: EdgeFlows_y_0_1 - EdgeFlows_y_1_4 - EdgeFlows_y_1_5 >= 0

_C2: EdgeFlows_y_0_1 - EdgeFlows_y_1_4 - EdgeFlows_y_1_5 <= 0

_C3: EdgeFlows_y_0_2 - EdgeFlows_y_2_4 - EdgeFlows_y_2_5 - EdgeFlows_y_2_6
 >= 0

_C4: EdgeFlows_y_0_2 - EdgeFlows_y_2_4 - EdgeFlows_y_2_5 - EdgeFlows_y_2_6
 <= 0

_C5: EdgeFlows_y_0_3 - EdgeFlows_y_3_5 - EdgeFlows_y_3_6 >= 0

_C6: EdgeFlows_y_0_3 - EdgeFlows_y_3_5 - EdgeFlows_y_3_6 <= 0

_C7: EdgeFlows_y_1_4 + EdgeFlows_y_2_4 - EdgeFlows_y_4_5 - E

In [21]:
capValues = [v.varValue for v in lp_problem.variables()[0:(len(edge_cap_vars))]]
cost = sum([cost_function(cap, 0) for cap in capValues])

14.0
